In [9]:
# Dependencies
import requests
import json
import pandas as pd
# Import the API key
from config import api_key
import csv

### Water consumption

In [10]:
# Dependencies and Setup
# Water consumption
import pandas as pd
from pathlib import Path

# File to Load (Remember to Change These)
path = Path("Resources/austin_water.csv")

customer_data_df = pd.read_csv(path)
customer_data_df

,Year Month,Postal Code,Customer Class,Total Gallons
0,201201,78613,Irrigation - Multi-Family,11000
1,201201,78613,Multi-Family,23000
2,201201,78617,Residential,19840800
3,201201,78617,Multi-Family,2477000
4,201201,78652,Irrigation - Residential,38500
...,...,...,...,...
16815,202009,78758,Multi-Family,57603300
16816,202009,78759,Irrigation - Residential,114500
16817,202009,78759,Multi-Family,45024600
16818,202009,78759,Irrigation - Multi-Family,3684700


#### Clean Data

#### Total water consumption by month

In [11]:
customer_data_df['Year Month'] = pd.to_datetime(customer_data_df['Year Month'], format='%Y%m')

customer_monthly_total= customer_data_df.groupby('Year Month',as_index=False)['Total Gallons'].sum()
customer_monthly_total

,Year Month,Total Gallons
0,2012-01-01,1729505700
1,2012-02-01,1512253200
2,2012-03-01,1648245943
3,2012-04-01,1766869789
4,2012-05-01,2212481568
...,...,...
100,2020-05-01,2009850900
101,2020-06-01,2343426200
102,2020-07-01,2693843700
103,2020-08-01,2811097800


# Weather Dataframe

In [12]:

import pandas as pd
from pathlib import Path

# File to Load (Remember to Change These)
path = Path("Resources/austin_weather.csv")

weather_data_df = pd.read_csv(path)
weather_data_df

,Date,TempHighF,TempAvgF,TempLowF,DewPointHighF,DewPointAvgF,DewPointLowF,HumidityHighPercent,HumidityAvgPercent,HumidityLowPercent,...,SeaLevelPressureAvgInches,SeaLevelPressureLowInches,VisibilityHighMiles,VisibilityAvgMiles,VisibilityLowMiles,WindHighMPH,WindAvgMPH,WindGustMPH,PrecipitationSumInches,Events
0,2013-12-21,74,60,45,67,49,43,93,75,57,...,29.68,29.59,10,7,2,20,4,31,0.46,"Rain , Thunderstorm"
1,2013-12-22,56,48,39,43,36,28,93,68,43,...,30.13,29.87,10,10,5,16,6,25,0,
2,2013-12-23,58,45,32,31,27,23,76,52,27,...,30.49,30.41,10,10,10,8,3,12,0,
3,2013-12-24,61,46,31,36,28,21,89,56,22,...,30.45,30.3,10,10,7,12,4,20,0,
4,2013-12-25,58,50,41,44,40,36,86,71,56,...,30.33,30.27,10,10,7,10,2,16,T,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1314,2017-07-27,103,89,75,71,67,61,82,54,25,...,29.97,29.88,10,10,10,12,5,21,0,
1315,2017-07-28,105,91,76,71,64,55,87,54,20,...,29.9,29.81,10,10,10,14,5,20,0,
1316,2017-07-29,107,92,77,72,64,55,82,51,19,...,29.86,29.79,10,10,10,12,4,17,0,
1317,2017-07-30,106,93,79,70,68,63,69,48,27,...,29.91,29.87,10,10,10,13,4,20,0,


#### Weather group by Average temp

In [13]:
# Group by 'Date' and calculate the sum of 'TempAvgF'
weather_data_df['Date'] = weather_data_df['Date'].str[:-3]
weather_monthly_total= weather_data_df.groupby('Date',as_index=False)['TempAvgF'].sum()
weather_monthly_total


,Date,TempAvgF
0,2013-12,537
1,2014-01,1577
2,2014-02,1545
3,2014-03,1831
4,2014-04,2082
5,2014-05,2319
6,2014-06,2481
7,2014-07,2622
8,2014-08,2708
9,2014-09,2424


### Merge